Eli Faigle
Recreating Figures 17 and 18 using AWS dataset
Goal of code:
1. Pull data from AWS dataset
2. Bin DENSITY of SST measurements
3. Create figures with SST measurement density's in half-degree spatial bins
4. Create histogram with 10 day binned counts of SST total, SST diurnal, and position and velo measurements

In [1]:
#Used for data manipulation
import numpy as np
import xarray as xr
from scipy import stats
#Others
import time
from datetime import datetime
import dask
from tqdm import tqdm
import re
#Data visualization
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.ticker import MaxNLocator
import cartopy.crs as ccrs
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import cartopy.feature as cfeature
import cmocean


In [2]:
subset_nb_drifters = 500  # For 2 GB, Not sure how to incorporate here yet

In [3]:
#Time to get the data from AWS
file = 'gdp_v2.00.nc'
url_path = 'https://noaa-oar-hourly-gdp-pds.s3.amazonaws.com/latest/'
ds = xr.open_dataset(url_path+file+'#mode=bytes')

c:\Users\19082\miniconda3\lib\site-packages\xarray\backends\plugins.py:71: RuntimeWarning: Engine 'cfgrib' loading failed:
Cannot find the ecCodes library
  warnings.warn(f"Engine {name!r} loading failed:\n{ex}", RuntimeWarning)


In [4]:
ds

<xarray.Dataset>
Dimensions:                (traj: 17324, obs: 165754333)
Coordinates:
    ids                    (obs) int64 ...
    time                   (obs) datetime64[ns] ...
    lon                    (obs) float32 ...
    lat                    (obs) float32 ...
Dimensions without coordinates: traj, obs
Data variables: (12/55)
    ID                     (traj) int64 ...
    rowsize                (traj) int32 ...
    WMO                    (traj) int32 ...
    expno                  (traj) int32 ...
    deploy_date            (traj) datetime64[ns] ...
    deploy_lat             (traj) float64 ...
    ...                     ...
    err_sst1               (obs) float64 ...
    err_sst2               (obs) float64 ...
    flg_sst                (obs) int8 ...
    flg_sst1               (obs) int8 ...
    flg_sst2               (obs) int8 ...
    drogue_status          (obs) bool ...
Attributes: (12/16)
    title:             Global Drifter Program hourly drifting buoy collection
    history:           Version 2.00.  Metadata from dirall.dat and deplog.dat
    Conventions:       CF-1.6
    date_created:      2022-12-09T06:02:29.684949
    publisher_name:    GDP Drifter DAC
    publisher_email:   aoml.dftr@noaa.gov
    ...                ...
    contributor_name:  NOAA Global Drifter Program
    contributor_role:  Data Acquisition Center
    institution:       NOAA Atlantic Oceanographic and Meteorological Laboratory
    acknowledgement:   Elipot, Shane; Sykulski, Adam; Lumpkin, Rick; Centurio...
    summary:           Global Drifter Program hourly data
    doi:               10.25921/x46c-3620

In [5]:
ds.rowsize

<xarray.DataArray 'rowsize' (traj: 17324)>
[17324 values with dtype=int32]
Dimensions without coordinates: traj
Attributes:
    long_name:         Number of observations per trajectory
    sample_dimension:  obs
    units:             -

NOTES TO REMEMBER ON XARRAY DATASETS

.ds commands to remember:
ds.___.isel = selecting data when plotting
EX: ds.air.isel(time=0).plot(x="lon");   ds.air.mean(dim="time").plot(x="lon")
ds.___.dims = see components/dimensions of array

Xarray supports:
label-based indexing using .sel
position-based indexing using .isel


# seasonal groups
ds.groupby("time.season")
DatasetGroupBy, grouped over 'season'
4 groups with labels 'DJF', 'JJA', 'MAM', 'SON'.
# make a seasonal mean
seasonal_mean = ds.groupby("time.season").mean()
seasonal_mean

In [6]:
traj_idx = np.insert(np.cumsum(ds.rowsize.values), 0, 0)

In [7]:
n = 2578 # What are the other IDs?
j = int(np.where(ds.ID==n)[0])
display('drifter index for ID '+str(n)+' is '+str(j))
ds.sst1[slice(traj_idx[j], traj_idx[j+1])]
ds.lon
ds.lat
ds.obs

'drifter index for ID 2578 is 0'

<xarray.DataArray 'obs' (obs: 165754333)>
array([        0,         1,         2, ..., 165754330, 165754331, 165754332],
      dtype=int64)
Coordinates:
    ids      (obs) int64 ...
    time     (obs) datetime64[ns] ...
    lon      (obs) float32 ...
    lat      (obs) float32 ...
Dimensions without coordinates: obs

In [8]:
benchmark_times = np.zeros((3,3))  # 3 benchmark tests for the 3 different processing libraries

Approach: Tally sst measurements 1 by 1 for each drifter, storing/graphing coordiantites
OR Tally sst measurements in each geographical bin location 

In [9]:
t0 = time.time()

lon = np.linspace(-180, 180, 360*2)
lat = np.linspace(-90, 90, 180*2)

ret = stats.binned_statistic_2d(ds.lon, 
                                ds.lat,
                                ds.sst,
                                statistic=np.nanmean, 
                                bins=[lon, lat])

benchmark_times[0,0] = time.time() - t0

KeyboardInterrupt: 

In [ ]:
x_c = np.convolve(lon, [0.5, 0.5], mode='valid')
y_c = np.convolve(lat, [0.5, 0.5], mode='valid')

# get 1st and 99th percentiles of values to plot to get a useful range for the colorscale
sst1,sst2 = np.nanpercentile(ret.statistic.T,[1,99])

fig = plt.figure(dpi=150)
ax = fig.add_subplot(1,1,1,projection=ccrs.Robinson(central_longitude=-180))
cmap = cmocean.tools.crop(cmocean.cm.balance, sstmin=sst1, sstmax=sst2, pivot=0)
pcm = ax.pcolormesh(x_c, y_c, 
                    ret.statistic.T, 
                    cmap=cmap, 
                    transform=ccrs.PlateCarree(),
                    sstmin=sst1, sstmax=sst2)

# gridlines and labels
gl = ax.gridlines(color='k', linewidth=0.1, linestyle='-',
                  xlocs=np.arange(-180, 181, 60), ylocs=np.arange(-90, 91, 30),
                  draw_labels=True)
gl.top_labels = False
gl.right_labels = False

# add land and coastline
ax.add_feature(cfeature.LAND, facecolor='grey', zorder=1)
ax.add_feature(cfeature.COASTLINE, linewidth=0.25, zorder=1)
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="3%", pad=0.05, axes_class=plt.Axes)
cb = fig.colorbar(pcm, cax=cax);
cb.ax.set_ylabel('Density of temperature measurements');

NameError: name 'lon' is not defined